<a href="https://colab.research.google.com/github/KyeoRaeHan/NLP/blob/main/notebooks/nlp/bert_sequence_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비
1. 라이브러리 다운로드
2. 네이버 영화평과 긍부정 데이터를 다운로드합니다 (파일보기 +  새로고침 후 확인)

* 자체 데이터셋을 사용할 경우 내용과 카테고리가 각각 content와 label 열에 들어가는 파일(아래 예시 참조)로 dataset.xlsx로 저장 후 기존 파일을 덮어쓰기 하면 됩니다. 
* 엑셀파일의 label과 content의 순서는 상관없으나 label은 0부터 시작하는 숫자로 입력하면 좋습니다. 예를들어 카테고리가 4개면 label을 0, 1, 2, 3으로 표시해주세요.

```
label  content
1      영화가 재밌다.     
1      이 영화 추천해요.     
0      지루한 영화였습니다.
...  
```

In [1]:
!pip3 install -q transformers
!git clone https://github.com/kiyoungkim1/ReadyToUseAI

from ReadyToUseAI.src.nlp import make_sample_dataset, bert_sequence_classification
make_sample_dataset.nsmc(mode='test', text_only=False)  # mode: which datasets? 'train' or 'test'

     |████████████████████████████████| 3.5 MB 8.8 MB/s 
     |████████████████████████████████| 6.8 MB 62.4 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 74.5 MB/s 
     |████████████████████████████████| 895 kB 72.8 MB/s 
Cloning into 'ReadyToUseAI'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 197 (delta 101), reused 103 (delta 48), pack-reused 0
Receiving objects: 100% (197/197), 43.56 KiB | 14.52 MiB/s, done.
Resolving deltas: 100% (101/101), done.


# [Training] 
* 첨부된 샘플의 경우 약 40min 소요 (Tesla T4 GPU)
* min_sentence_length보다 긴 문장만 사용합니다.
* MAX_LEN은 모델이 인식하는 token의 길이로, 전체길이가 약 MAX_LEN의 2배보다 긴 문장은 뒷부분이 삭제됩니다 (예를들어 MAX_LEN = 128이면, 대략 길이가 256이상인 문장은 뒷부분이 무시됨).
* batch_size는 한번에 몇개의 sample을 계산하는지를 나타내며, 제한된 메모리에서 MAX_LEN을 줄이면 batch_size를 키울 수 있고, MAX_LEN를 키우면 batch_size를 줄여야 합니다. 
* epochs는 데이터셋을 몇번 반복해서 학습할지 여부이며, dataset_split은 전체 데이터 중 몇 %를 검증용 데이터셋으로 사용할지 여부입니다.

In [2]:
CLS = bert_sequence_classification.Classification(model_name='kykim/bert-kor-base', min_sentence_length=10, MAX_LEN=128, batch_size=32, use_bert_tokenizer=True)
CLS.dataset(data_path='dataset.xlsx')
CLS.load_model(mode='train')
CLS.train(epochs=3, dataset_split=0.1)

2 labels, 44682 dataset
label counts:: Counter({1: 22512, 0: 22170})


Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/454M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

train-dataset is prepared

======== Epoch 1 / 3 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  Batch   100  of  1,257.    Elapsed: 0:00:39.
  Batch   200  of  1,257.    Elapsed: 0:01:17.
  Batch   300  of  1,257.    Elapsed: 0:01:56.
  Batch   400  of  1,257.    Elapsed: 0:02:34.
  Batch   500  of  1,257.    Elapsed: 0:03:13.
  Batch   600  of  1,257.    Elapsed: 0:03:52.
  Batch   700  of  1,257.    Elapsed: 0:04:30.
  Batch   800  of  1,257.    Elapsed: 0:05:09.
  Batch   900  of  1,257.    Elapsed: 0:05:47.
  Batch 1,000  of  1,257.    Elapsed: 0:06:26.
  Batch 1,100  of  1,257.    Elapsed: 0:07:04.
  Batch 1,200  of  1,257.    Elapsed: 0:07:43.

  Train loss: 0.31, Train Accuracy: 0.87
  Training epcoh took: 0:08:05

Running Validation...
  Validation Accuracy: 0.89
  Validation took: 0:00:17
              precision    recall  f1-score   support

           0       0.86      0.92      0.89      2177
           1       0.92      0.85      0.88      2292

    accuracy                           0.89      4469
   macro avg       0.89      0.89      0.89      4469
weighted avg 

# [Inference]
* sentences에 원하는 문장을 아래 형식과 같이 넣으면 해당하는 카테고리를 반환합니다.
* saved_model_path는 학습된 모델이 저장된 '폴더명'입니다.

In [3]:
sentences = ['영화 재밌어요', '영화 재미없어요', '그냥 시간떼우기용', '완전 추천작']
saved_model_path='model/saved/3'

CLS = bert_sequence_classification.Classification(model_name='kykim/bert-kor-base', min_sentence_length=10, MAX_LEN=128, batch_size=64, use_bert_tokenizer=True)
CLS.load_model(mode='inference', saved_model_path=saved_model_path)
logit = CLS.inference(sentences=sentences)
print(logit)  # 네이버 영화평의 경우 0은 부정 카테고리, 1은 긍정 카테고리로 설정되어 있음

[1, 0, 0, 1]
